In [42]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [43]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60

    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0
        # 2 -1 = 1 temperature
        self.state += action -1
        # Reduce shower length by 1 second
        self.shower_length -= 1

        # Calculate reward
        if self.state >=37 and self.state <=39:
            reward =1
        else:
            reward = -1

            # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False

        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60
        return self.state


In [44]:
env = ShowerEnv()

C:\Users\elias\anaconda3\lib\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [45]:
env.observation_space.sample()

array([77.85204], dtype=float32)

In [46]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-26
Episode:2 Score:-52
Episode:3 Score:-24
Episode:4 Score:-34
Episode:5 Score:-60
Episode:6 Score:4
Episode:7 Score:-20
Episode:8 Score:-56
Episode:9 Score:-10
Episode:10 Score:-22


In [58]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [59]:
states = env.observation_space.shape
actions = env.action_space.n

In [60]:
actions

3

In [61]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [62]:
model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [63]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))
_ = dqn.test(env, nb_episodes=15, visualize=True)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 7:10 - reward: -1.0000

C:\Users\elias\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\elias\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
C:\Users\elias\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   49/10000 [..............................] - ETA: 1:50 - reward: -1.0000

C:\Users\elias\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\elias\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\elias\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\elias\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\elias\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) 

10000/10000 [==============================] - 39s 4ms/step - reward: -0.5330
166 episodes - episode_reward: -32.169 [-60.000, 32.000] - loss: 1.377 - mae: 7.795 - mean_q: -10.969

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: -0.5126
167 episodes - episode_reward: -30.599 [-60.000, 30.000] - loss: 2.242 - mae: 10.616 - mean_q: -15.153

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 42s 4ms/step - reward: -0.5060
167 episodes - episode_reward: -30.335 [-60.000, 40.000] - loss: 2.653 - mae: 11.739 - mean_q: -16.847

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 42s 4ms/step - reward: -0.4886
166 episodes - episode_reward: -29.361 [-60.000, 46.000] - loss: 2.705 - mae: 11.738 - mean_q: -16.854

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: -0.4732
done, took 209.016 seconds
Testing for 100 episodes ...

TypeError: render() got an unexpected keyword argument 'mode'